In [1]:
import sys 
sys.path.append('../src') 
import numpy as np
#for some reason, matplotlib crashes without these lines
a = np.zeros((5,5))
a@a
import matplotlib.pyplot as plt
from utils import get_mesh_vtk
from utils import get_data
from utils import strip_cross, reconstruct_cross
from KPCA import *
import KPCA
import Autoencoder
# from torchsummary import summary
# from Autoencoder import Autoencoder
import torch
import torch.nn as nn
from utils import calc_energy, torch2np
import glob
import os
# from importlib import reload

ModuleNotFoundError: No module named 'utils'

In [ ]:
def get_t(filename):
    str_s = filename.split('_')[4].replace('t','')
    t = float(str_s)
    return t

def read_from_ds(Wi, beta, ds_path=None):
    if ds_path is None:
        ds_path = '../../npz_data/four_roll_train'

    files = glob.glob(f'*Wi{Wi:g}_beta{beta:g}_*.pt', root_dir=ds_path)
    X = torch.load(f'{ds_path}/{files[0]}')['tensor']
    P = torch.load(f'{ds_path}/{files[0]}')['param']
    data = torch.zeros([len(files)] + [s for s in X.shape])
    params = torch.zeros([len(files)] + [len(P)])


    files = np.array(files)
    times = [get_t(fname) for fname in files]
    idx = np.argsort(times)
    files = files[idx]
    for i,file in enumerate(files):
        data_file = torch.load(f'{ds_path}/{file}')
        data[i] = data_file['tensor'].float()
        params[i] = torch.tensor(data_file['param']).float()
    return data, params

def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

In [ ]:
parameters = [[2.0, 0.11111],
 [2.0, 0.4],
 [2.0, 0.75],
 [3.0, 0.11111],
 [3.0, 0.4],
 [3.0, 0.75],
 [3.5, 0.11111],
 [3.5, 0.4],
 [3.5, 0.75],
 [3.8, 0.11111],
 [3.8, 0.4],
 [3.8, 0.75]]

In [ ]:
dspath = '../../npz_data/four_roll_train_osc'
Xpca = np.zeros((5*4096,0))
Parampca = np.zeros((0,2))
for p in parameters:
    X_torch, param = read_from_ds(*p, ds_path=dspath)

    Xpca = np.concatenate((Xpca,torch2np(X_torch)), axis=1)
    Parampca = np.concatenate((Parampca,torch2np(param)), axis=0)
    break